# Flame

In order to download the models we need to look at: https://flame.is.tue.mpg.de/download.php
This could be usefull if onw want's to look how to load the FLAME model from the SMPL loader: https://github.com/Rubikplayer/flame-fitting/blob/master/smpl_webuser/serialization.py#L117
Useful utils if one needs to transform the chumpy format into nupy or torch: https://github.com/vchoutas/smplx/blob/main/smplx/utils.py

NOTE: That if one want't to unpickle old python=2.x numpy code, we need to use the encoding="latin1". For more information please refere to: https://docs.python.org/3/library/pickle.html


In [ ]:
import pickle
from pathlib import Path
import numpy as np
import torch
import warnings


def to_tensor(array, dtype=torch.float32):
    if torch.is_tensor(array):
        return array
    return torch.tensor(array, dtype=dtype)


def to_np(array, dtype=np.float32):
    if "scipy.sparse" in str(type(array)):
        array = array.todense()
    return np.array(array, dtype=dtype)


def load_flame_2023_no_jaw(flame_dir: str | Path):
    # https://github.com/soubhiksanyal/FLAME_PyTorch/blob/master/requirements.txt
    # the requirenemnts needs to be fixed and we need to have chumpy installed
    # also python 3.10 is required with an "older" numpy version
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    path = Path(flame_dir) / "flame2023_no_jaw/flame.pkl"
    with open(path, "rb") as f:
        return pickle.load(f, encoding="latin1")


def load_flame_2023(flame_dir: str | Path):
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    path = Path(flame_dir) / "flame2023/flame.pkl"
    with open(path, "rb") as f:
        return pickle.load(f, encoding="latin1")


# https://github.com/soubhiksanyal/FLAME_PyTorch/blob/master/flame_pytorch/flame.py
flame_dir = "/Users/robinborth/Code/GuidedResearch/checkpoints"
flame_model = load_flame_2023(flame_dir)
print("FLAME keys:")
print(list(flame_model.keys()))
print()

# This is the linear blend skinning (LBS) with corrective blendshapes with N=5023 and
# K=4 joint (neck, jaw, and eyeballs (left, right))
bs_style = flame_model["bs_style"]
print("bs_style:", bs_style)
bs_type = flame_model["bs_type"]
print("bs_type:", bs_type)

# this is the template mesh, e.g. T bar in the "zero pose"
v_template = to_np(flame_model["v_template"])
print("v_template:", v_template.shape)

# those are used in the pytorch flame example
f = to_np(flame_model["f"])
print("f:", f.shape)

# shape (beta); note that the dimension is (5023, 3, 400)
# where the first 300 are for the shape params and the last 400 for the expression
# params, but the matrix is shared
shapedirs = to_np(flame_model["shapedirs"])
print("shapedirs:", shapedirs.shape)

# pose (theta)
posedirs = to_np(flame_model["posedirs"])
print("posedirs:", posedirs.shape)

# is this the expressions? (psi)
weights = to_np(flame_model["weights"])  # lbs := linear blend shapes
print("weights:", weights.shape)

# Linear smoothed by skinning function(T, J, theta, W).
# Blendweights W (KxN) are J_regressor
J_regressor = to_np(flame_model["J_regressor"])
print("J_regressor:", J_regressor.shape)

# J are the joints that the vertices of T are rotated
J = to_np(flame_model["J"])
print("J:", J.shape)

kintree_table = to_np(flame_model["kintree_table"])
print("kintree_table:", kintree_table.shape)

In [ ]:
from lib.flame.utils import load_flame

flame_dir = "/Users/robinborth/Code/GuidedResearch/checkpoints/flame2023"
flame = load_flame(flame_dir=flame_dir)

print("faces:", flame["f"].shape)
print(flame["f"].min())
print(flame["f"].max())

# Falme Landmarks

The landmark file defines the barycentric embedding of 105 points of the Mediapipe mesh in the surface of FLAME.
In consists of three arrays: lmk_face_idx, lmk_b_coords, and landmark_indices.

- lmk_face_idx contains for every landmark the index of the FLAME triangle which each landmark is embedded into
- lmk_b_coords are the barycentric weights for each vertex of the triangles
- landmark_indices are the indices of the vertices of the Mediapipe mesh


In [ ]:
def load_flame_landmarks(flame_dir: str | Path):
    path = Path(flame_dir) / "mediapipe_landmark_embedding.npz"
    return np.load(path)


flame_dir = "/Users/robinborth/Code/GuidedResearch/checkpoints/flame2023"
flame_landmarks = load_flame_landmarks(flame_dir)
print(list(flame_landmarks.keys()))
print()

print("lmk_face_idx:")
print(flame_landmarks["lmk_face_idx"][:5])
print(flame_landmarks["lmk_face_idx"].min())
print(flame_landmarks["lmk_face_idx"].max())
print(flame_landmarks["lmk_face_idx"].shape)
print()

print("lmk_b_coords:")
print(flame_landmarks["lmk_b_coords"][:5])
print(flame_landmarks["lmk_b_coords"].min())
print(flame_landmarks["lmk_b_coords"].max())
print(flame_landmarks["lmk_b_coords"].shape)
print()

print("landmark_indices:")
print(flame_landmarks["landmark_indices"][:5])
print(flame_landmarks["landmark_indices"].min())
print(flame_landmarks["landmark_indices"].max())
print(flame_landmarks["landmark_indices"].shape)

In [ ]:
# called normalized or absolute barycentric coordinates
# hence they are unique then
flame_landmarks["lmk_b_coords"].sum(-1)

# FLAME Mask

Dictionary with vertex indices for different masks for the publicly available FLAME head model (https://flame.is.tue.mpg.de/).
See the gif for a visualization of all masks.


In [ ]:
def load_flame_masks(flame_dir: str | Path):
    path = Path(flame_dir) / "FLAME_masks.pkl"
    with open(path, "rb") as f:
        return pickle.load(f, encoding="latin1")


flame_masks = load_flame_masks(flame_dir)
print(list(flame_masks.keys()))
print()

print("eye_region:")
print(flame_masks["eye_region"][:5])
print(flame_masks["eye_region"].min())
print(flame_masks["eye_region"].max())
print(flame_masks["eye_region"].shape)

# Render the flame model


In [ ]:
import trimesh
import pyrender
import numpy as np

path = "/Users/robinborth/Code/GuidedResearch/data/flame_registrations/d3dfacs_alignments/Adrian/1+2/1+2_120.ply"
mesh = trimesh.load_mesh(path)
vertex_colors = np.ones([mesh.vertices.shape[0], 4]) * [0.3, 0.3, 0.3, 0.8]
mesh = trimesh.Trimesh(mesh.vertices, mesh.faces, vertex_colors=vertex_colors)

scene = pyrender.Scene()
scene.add(pyrender.Mesh.from_trimesh(mesh))
pyrender.Viewer(scene=scene, use_raymond_lightning=True)

# FLAME


In [ ]:
from lib.flame import FLAME

flame_dir = "/Users/robinborth/Code/GuidedResearch/checkpoints/flame2023"
flame = FLAME(flame_dir=flame_dir)

In [ ]:
import torch
import numpy as np
import trimesh

# forward the model
shape_params = torch.zeros(300)
expression_params = torch.zeros(100)
vertices = flame(
    shape_params=shape_params,
    expression_params=expression_params,
)
vertices = vertices.detach().cpu().numpy()

# convert to trimesh
vertex_colors = np.ones([vertices.shape[0], 4]) * [0.3, 0.3, 0.3, 0.8]
mesh = trimesh.Trimesh(vertices, flame.faces, vertex_colors=vertex_colors)

# Rendering

In [ ]:
from pyrender import OffscreenRenderer, Mesh, Scene, PerspectiveCamera
import matplotlib.pyplot as plt

# add mesh to scene
scene = Scene()
scene.add(Mesh.from_trimesh(mesh))

# create camera
cam = PerspectiveCamera(yfov=(np.pi / 3.0))
cam_pose = np.array(
    [
        [1.0, 0.0, 0.0, 0.0],
        [0.0, 1.0, 0.0, 0.0],
        [0.0, 0.0, 1.0, 0.5],
        [0.0, 0.0, 0.0, 1.0],
    ]
)
cam_node = scene.add(cam, pose=cam_pose)

# render the scene
renderer = OffscreenRenderer(viewport_width=1024, viewport_height=1024)
color, depth = renderer.render(scene)
renderer.delete()
plt.figure(figsize=(20,20))
plt.imshow(color)
plt.show()

# Gaussian Head FLAME

In [ ]:
import numpy as np

path = "/Users/robinborth/Code/GuidedResearch/checkpoints/gaussianhead/landmark_embedding_with_eyes.npy"
lms = np.load(path, allow_pickle=True)
# TODO inspect lms